In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline

In [8]:
plt.rc('figure',dpi=100);

In [9]:
postulantes_ge = pd.read_csv("Data/fiuba_entrenamiento/postulantes_ge.csv")
postulantes_ge.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [10]:
postulantes_educacion = pd.read_csv("Data/fiuba_entrenamiento/postulantes_educacion.csv")
postulantes_educacion.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [11]:
postulantes_ge['fechanacimiento'] = pd.to_datetime(postulantes_ge['fechanacimiento'], errors = 'coerce')
postulantes_ge['edad'] = datetime.now() - postulantes_ge['fechanacimiento']
postulantes_ge['edad']=pd.TimedeltaIndex(postulantes_ge['edad']).days;
postulantes_ge['edad']=np.floor(postulantes_ge['edad']/365);
postulantes_ge.drop(['fechanacimiento'],axis=1, inplace=True);
postulantes_ge.head()

,idpostulante,sexo,edad
0,NM5M,FEM,47.0
1,5awk,FEM,55.0
2,ZaO5,FEM,39.0
3,NdJl,MASC,49.0
4,eo2p,MASC,37.0


In [12]:
def mayor_estado(estados):
    retval="Abandonado"
    for elem in estados:
        if(elem=="En Curso"): retval=elem;
        if(elem=="Graduado"): retval=elem;
    return retval
group = postulantes_educacion.groupby(['idpostulante','nombre'], as_index=False).agg({'estado':mayor_estado})
group.head()

,idpostulante,nombre,estado
0,0z5Dmrd,Universitario,Graduado
1,0z5JW1r,Otro,Graduado
2,0z5JW1r,Secundario,Graduado
3,0z5JW1r,Terciario/Técnico,Graduado
4,0z5JW1r,Universitario,Abandonado


In [13]:
group2=group.pivot_table(values='estado', index=['idpostulante'], columns='nombre', aggfunc='first').reset_index()
group2=group2.fillna("-")
group2.head()

nombre,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0z5Dmrd,-,-,-,-,-,-,Graduado
1,0z5JW1r,-,-,Graduado,-,Graduado,Graduado,Abandonado
2,0z5VvGv,-,-,-,-,Graduado,-,-
3,0zB01pE,-,-,-,-,-,Graduado,Abandonado
4,0zB026d,-,-,-,-,Graduado,Graduado,-


In [14]:
postulantes=pd.merge(postulantes_ge,group2, on='idpostulante',how='left');
postulantes.fillna("-")
postulantes.head()

,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,NM5M,FEM,47.0,-,-,-,-,Graduado,-,-
1,5awk,FEM,55.0,-,-,-,-,-,Graduado,Graduado
2,ZaO5,FEM,39.0,-,-,-,-,-,Graduado,Abandonado
3,NdJl,MASC,49.0,-,-,-,En Curso,-,-,Graduado
4,eo2p,MASC,37.0,-,Graduado,-,-,Graduado,-,Graduado


In [15]:
postulantes.edad=postulantes.edad.fillna(200)
postulantes=postulantes.fillna("-")
postulantes=postulantes.replace("NaN", "-")
postulantes.sample(20)

,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
182821,NzrKwrx,MASC,19.0,-,-,-,-,Graduado,-,-
283393,vV3z1p1,FEM,23.0,-,-,-,-,-,-,Graduado
255843,5mrPmXZ,FEM,35.0,-,-,-,-,En Curso,-,-
42350,akbD0pJ,FEM,28.0,-,-,-,-,Graduado,-,Graduado
468251,DrXZ6LZ,NO_DECLARA,200.0,-,-,-,-,-,-,-
255958,GNJZVRA,MASC,30.0,-,-,-,-,Graduado,-,Graduado
422017,lDwlA2X,MASC,28.0,-,-,-,-,Graduado,Graduado,-
25115,ErZPpz,FEM,26.0,-,-,-,-,Graduado,En Curso,-
490510,Yj5bDJV,MASC,30.0,-,-,-,-,Graduado,-,En Curso
418939,lDLemBj,FEM,26.0,-,-,-,-,-,-,Graduado


In [16]:
def completar_estudio_menor(estudio_menor,estudio_mayor):
    if(estudio_mayor=="Graduado"):
        return "Graduado"
    return estudio_menor;
completar_estudio_menor=np.vectorize(completar_estudio_menor)
postulantes['Secundario']=completar_estudio_menor(postulantes['Secundario'],postulantes['Universitario'])
postulantes.head()

,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,NM5M,FEM,47.0,-,-,-,-,Graduado,-,-
1,5awk,FEM,55.0,-,-,-,-,Graduado,Graduado,Graduado
2,ZaO5,FEM,39.0,-,-,-,-,-,Graduado,Abandonado
3,NdJl,MASC,49.0,-,-,-,En Curso,Graduado,-,Graduado
4,eo2p,MASC,37.0,-,Graduado,-,-,Graduado,-,Graduado


In [17]:
print(postulantes.shape)

(505382, 10)


In [18]:
postulantes.drop_duplicates(['idpostulante'],keep='first',inplace=True)
postulantes.to_csv("Data/fiuba_entrenamiento/pocho/postulantes.csv",index=False)